# Data retrieval

> Module die de benodigde peilbuis gegevens ophaalt uit de Azure database en voorbewerkt voor het berekenen van de optimale routes.

In [ ]:
#| default_exp data_get

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastcore.utils import Path
import yaml
import pickle

from WDODAzureSQL.azure_connector_local import AzureSQLDBConnector
from project.utils import get_project_root, funcresults_to_pickle, load_settings

## Get settings from the settings.yaml file

In [ ]:
#| export
settings = load_settings()

## Get data from the Azure SQL database and place it in a pandas DataFrame.

Optionaly store the DataFrame in a pickle file.

In [ ]:
#| export

@funcresults_to_pickle(settings['files']['path_pickle_results'])
def get_data_from_azuresql(sql_statement=None, save_pickle=False):
    """
    Gets data from an Azure SQL database and returns a pandas DataFrame.

    Parameters
    ----------
    jdbc_host_name : str
        The host name of the Azure SQL database. Retrieved from settings.yaml
    jdbc_database : str
        The name of the Azure SQL database. Retrieved from settings.yaml
    meta_runinfo_table : str, optional
        The name of the table containing run info. If None, then no run info is read.
        Retrieved from settings.yaml
    sql_statement : str
        The SQL statement to read in the dimensions of the data. Can be given
        as parameter, if none is given, will be retrieved from settings.yaml

    Returns
    -------
    peilbuizen_df : pandas.DataFrame
        A pandas DataFrame containing the locations, latest measurement date and project
        of the peilbuizen
    """


    if not sql_statement:
        sql_statement = settings['sql_statement']['peilbuizen']

    print(f"\nMake connection with Azure SQL {settings['azure']['jdbc_hostname']}\n")
    db_connection = AzureSQLDBConnector(jdbc_host_name=settings['azure']['jdbc_hostname'],
                                    jdbc_database=settings['azure']['jdbc_database'],
                                    jdbc_port="1433",
                                    meta_runinfo_table=None)

    peilbuizen_df = db_connection.read_df(sql_statement)
    print(f"\nRetrieved dimensions from database {settings['azure']['jdbc_database']}.\n")

    return peilbuizen_df

## Get data from a stored pickle file

The pickle should contain the results that were retrieved and stored using the `get_data_from_azuresql` function.

In [ ]:
#| export

def load_pickle(file_path):
    with open(file_path, 'rb') as f:
        obj = pickle.load(f)

    return obj

In [ ]:
#| hide
# This is only used to test
file_path = settings['files']['path_pickle_results']
file_name = "get_data_from_azuresql_20250124_130111.pickle"
peilbuizen_df = load_pickle(file_path=Path(file_path) / file_name)
peilbuizen_df.head()

,Id,Longitude,latitude,latest_measure_date,project
0,21GG512A;1058864,6.135953,52.711753,2025-01-08 13:13:41,None
1,17BC312A;1071534,6.554916,52.820904,2024-09-12 09:46:29,Oude Diep Mantinge Bos_Zand
2,WA258.207;29347,6.440575,52.794306,2024-08-29 07:47:40,Wabos-KRW
3,Veenweidepolders-PB14;29882,5.901383,52.803128,2024-06-17 10:48:27,Primair meetnet blok 3
4,MIDDENLOOP-PB5;29752,6.256332,52.884036,2024-04-16 07:29:57,Ecologisch Effect Beekherstel Middenloop Vledd...


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()